In [1]:
wcs_api_key = '6HdTpfDmEaUNqSJ1FRz9A9aaezvxp4PyPlBw'
palm_api_key = 'ya29.a0Ad52N39LVK82gQOddk8PDTYoI7ls8yQFJGSawGTzBEf7I2hZJdRGmihiyLUjATN-j4ngMZpmPeccB1H9OUT92yvM-IvQUeL3SVUPdgA35Mk8rZ6lAVRKCpgxnirug_4ZvpLvPUpwXWqVHxrjrC20Mld1RiVh_7ieW3v9-RaiGAaCgYKAUsSARMSFQHGX2MickWKMOuWA125Nl8ZrkrBWg0177'
cluster_url = 'https://ridedott-hackathon-8nl5t484.weaviate.network'
gcloud_project_id = 'qwiklabs-gcp-02-08a8c713b151'

In [2]:
import weaviate
from weaviate.classes.config import Property, DataType


client = weaviate.connect_to_wcs(
    cluster_url=cluster_url,
    auth_credentials=weaviate.auth.AuthApiKey(wcs_api_key),
    headers={"X-PaLM-Api-Key": palm_api_key},
)

In [3]:
import weaviate.classes as wvc

collection = client.collections.create(
    name="instructions_collection",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_palm(
        project_id=gcloud_project_id,
        model_id="text-embedding-preview-0409",
    ),
    properties=[
        wvc.config.Property(name="filename", data_type=wvc.config.DataType.TEXT),
        wvc.config.Property(name="content", data_type=wvc.config.DataType.TEXT),
    ],
)

In [4]:
# client.collections.delete("instructions_collection")

In [5]:
import os

directory_path = '/Users/teddy/Downloads/treatments'
with collection.batch.dynamic() as batch:
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):  # Ensures only .txt files are processed
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
            # print(filename)
            # print(file_content)
            
            batch.add_object(
                    properties={
                        "filename": filename,
                        "content": file_content,
                    },
                )

In [6]:
result = collection.query.hybrid("what is fertility", limit=10)

In [7]:
print(result)

QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('c5d8e4dc-7e98-46b4-8ee5-6b19bbf5abb1'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': "intro: Childhood cancer treatment can sometimes lead to infertility. There are options for boys who have not yet reached puberty. Those options are in a research setting. If your son is at risk of becoming infertile, you will have an interview with a fertility nurse specialist and a fertility doctor or urologist. If possible, this interview takes place before your son's treatment starts.\n\n\n\nWe know that being able to father children is very important to many adult men who have had childhood cancer. So you will have an indepth consultation with a clinical nurse specialist and/or a doctor who specializes in fertility. The following matters are discussed during this consultation:\n\nThe effect of the trea

In [8]:
for e in result.objects:
    print(e.properties.get("filename"))

fertility-preservation-in-boys.txt
germ-cell-tumors-in-the-body.txt
stem-cell-transplant-allogeneic.txt
fanconi-anemia-fa.txt
myelodysplastic-syndrome-mds.txt
germ-cell-tumor-of-the-brain.txt
ultrasound.txt
bone-marrow-failure.txt
bone-marrow-aspiration.txt
hemophagocytic-lymphohistiocytosis-hlh.txt
